## Experiment design

In [2]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import random
random.seed(10)

from policy import Policy

In [95]:
treat_data = pd.read_csv("treatment_features.csv",header=None)
action_data = pd.read_csv("treatment_actions.csv",header=None)
outcome_data = pd.read_csv("treatment_outcomes.csv",header=None)

symptom_names = ['Covid-Recovered', 'Covid-Positive', 'No-Taste/Smell', 'Fever', 'Headache', 
                  'Pneumonia', 'Stomach', 'Myocarditis', 'Blood-Clots', 'Death','Age', 'Gender', 'Income']
cols = ( symptom_names +
         [f'Gene_{i+1:03}' for i in range(128)] +
         ['Asthma', 'Obesity', 'Smoking', 'Diabetes', 'Heart disease', 'Hypertension',
         'Vacc_1', 'Vacc_2', 'Vacc_3'])

treat_data.columns = cols
outcome_data.columns = cols[:10]
action_data.columns = ['Treatment_1', 'Treatment_2']

## Estimaring the $P(y|x,a)$:

In [246]:
X = treat_data
y=Y = outcome_data
a = action_data 

X_tr,X_ts,Y_tr,Y_ts = train_test_split(X,Y,test_size=0.33,random_state=1)


In [408]:
class MLPModel(MLPClassifier):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def get_probabilities(self, features, action):
        features = features.assign(**{action:np.ones(features.shape[0])})

        return self.predict_proba(features)

In [405]:
class NewPolicy(Policy):
    def __init__(self, n_actions, action_set):
        super().__init__(n_actions, action_set)
        self.model = MLPModel(activation='logistic', #max_iter=10_000,
                           warm_start=True, random_state=0)
        
        self.weights = {'Covid-Recovered': 0, 
                         'Covid-Positive': .2, 
                         'No-Taste/Smell': .1, 
                         'Fever': .1, 
                         'Headache': .1,
                         'Pneumonia': .5,
                         'Stomach': .2, 
                         'Myocarditis': .5, 
                         'Blood-Clots': 1,
                         'Death': 100}
    
    def observe(self, features, action, outcome):
        features = features.assign(**{key:action[key] for key in action})
        self.model.fit(features, outcome)
        
    def get_utility(self, features, action, outcome):
        """ Treatment 2 is more 'costly' than treatment 1, so associate somewhat lower
        utility to treatment 2. Outcomes are weighted more highly than previous condition, 
        but it is better to have had a symptom and then remove it, than just never having 
        the symptom. 
        """
        utility = 0
        for key in action:
            utility -= sum(action[key]) * np.log(self.action_set.index(key) + 2)
            
        #actions = self.get_action(features)
        for key, weight in self.weights.items():
            utility += weight * sum(0.7 * features.loc[:, key] - outcome.loc[:, key])
        
        return utility
    
    def get_expected_utility(self, action, p):
        utility = np.zeros(p.shape[0])
        #utility -= np.log(self.action_set.index(action)+2)
    
        weights = np.array(list(self.weights.values()))
        
        utility -= (p @ weights.T)
        
        return utility
             
    
    def get_action(self, features):
        """ Get actions for one or more people. 
        Args: 
        features (t*|X| array)
        Returns: 
        actions (t*|A| array)
        """
        features = features.assign(**{key:np.zeros(features.shape[0])
                                      for key in self.action_set})
        res = []
        for action in (self.action_set):
            p = self.model.get_probabilities(features, action)
            res.append(self.get_expected_utility(action, p))

        return np.argmax(res, axis=0)
        
        
pol = NewPolicy(2, ['Treatment_1', 'Treatment_2'])

Initialising policy with  2 actions
A = { ['Treatment_1', 'Treatment_2'] }


In [410]:
pol.observe(X_tr, a[a.index.isin(Y_tr.index)], Y_tr)

In [411]:
acts = pol.get_action(X_ts)

In [412]:
sum(acts), acts.shape

(1075, (3353,))

In [413]:
pol.get_utility(X_ts, a[a.index.isin(Y_ts.index)], Y_ts, )

281.01670360944445

In [414]:
pol.get_utility(X_tr, a[a.index.isin(Y_tr.index)], Y_tr, )

-185.85376017977342

In [415]:
pol.get_utility(X, a, Y)

95.16294342957735

In [403]:
X1 = X[a['Treatment_1'] == 1]
X2 = X[a['Treatment_2'] == 1]

#y1 = pd.DataFrame([Y.iloc[i] for i in range(len(a)) if a.iloc[i][0] == 1])
#y2 = pd.DataFrame([Y.iloc[i] for i in range(len(a)) if a.iloc[i][1] == 1])

y1 = y[a['Treatment_1'] == 1]
y2 = y[a['Treatment_2'] == 1]

## Utility

In [6]:
def get_utility(features, action, outcome):
    """Here the utiliy is defined in terms of the outcomes obtained only, ignoring both the treatment and the previous condition.
    """
    """for i, symptom in enumerate(symptom_names):
    utility[I] = weights[symptom] * outcome[symptom] """
    
    utility = np.zeros(10)

    utility[0] -= 0
    utility[1] -= 0
    utility[2]  -= 0.1 * sum(outcome['No-Taste/Smell'])
    utility[3]  -= 0.1 * sum(outcome['Fever'])
    utility[4]  -= 0.1 * sum(outcome['Headache'])
    utility[5]  -= 0.5 * sum(outcome['Pneumonia'])
    utility[6]  -= 0.2 * sum(outcome['Stomach'])
    utility[7]  -= 0.5 * sum(outcome['Myocarditis'])
    utility[8]  -= 1.0 * sum(outcome['Blood-Clots'])
    utility[9]  -= 100.0 * sum(outcome['Death'])
    
    return utility

## Estimating:
$$\sum P(y|x,a) \; u(a,y)$$

In [7]:
u1 = get_utility(None,None,y1)
u2 = get_utility(None,None,y2)

In [8]:
y2_proba = train_pred(X2,y2)
y1_proba = train_pred(X1,y1)

In [9]:
expected_utility1 = np.array([u*y for u,y in zip(u1,y1_proba)])
expected_utility2 = np.array([u*y for u,y in zip(u2,y2_proba)])

In [10]:
y1_proba.T @ u1

-5.786567040926596

In [11]:
y2_proba.T @ u2

-750.9740762834626

In [12]:
sum(expected_utility1)/len(X1)

-0.0011556954345769113

In [13]:
sum(expected_utility2)/len(X2)

-0.1457353146290438

### Defining the utility

In [14]:
#finding optimal action for given symptoms for each individual

optimal_action = np.zeros([X.shape[0],2])
for i in range(X.shape[0]):
    index = np.argwhere(np.array(X.iloc[i,2:10]) == np.ones(8)).flatten().tolist()
    
    if len(index) > 0:
        if np.sum(expected_utility1[index]) >= np.sum(expected_utility2[index]):
            optimal_action[i,0] = 1
        else:
            optimal_action[i,1] = 1

## Bootstrap the data for:
Provide error bounds on the expected utility and explain how those were obtained.

## Expected utility for improved policy

In [15]:
X1_improved = pd.DataFrame([X.iloc[i] for i in range(len(optimal_action)) if optimal_action[i][0] == 1])
X2_improved = pd.DataFrame([X.iloc[i] for i in range(len(optimal_action)) if optimal_action[i][1] == 1])

In [276]:
#taken from "simulator.py" file

def treatment(X,p):
    A = np.random.uniform(size=[2,8])
    treatments = np.zeros([X.shape[0], 2])
    result = np.zeros([X.shape[0], 8])
    for t in range(X.shape[0]):
        treatments[t] = optimal_action[t]
        r = np.array(np.matrix(treatments[t]) * A).flatten()
        for k in range(8):
            result[t,k] = np.random.choice(2,p=[1-p[k],p[k]])
        ##print("X:", X[t,:self.n_symptoms] , "Y:",  result[t])
    return treatments, result

In [295]:
p_symptoms1 = np.zeros(8)
p_symptoms2 = np.zeros(8)
for i in range(8):
    p_symptoms1[i] = y1.iloc[:,i+2].mean()
    p_symptoms2[i] = y2.iloc[:,i+2].mean()

In [304]:
Y1_new = pd.DataFrame(treatment(X1_improved.iloc[:,2:10],y1_proba)[1], columns = symptom_names[2:10])
Y2_new = pd.DataFrame(treatment(X2_improved.iloc[:,2:10],y2_proba)[1], columns = symptom_names[2:10])

In [305]:
new_action = pd.DataFrame({'Treatment1':optimal_action[:,0],'Treatment2':optimal_action[:,1]})

In [306]:
#doing the same as above for computing the expected utility

u1_new = get_utility(None,None,Y1_new)
u2_new = get_utility(None,None,Y2_new)

y2_new_proba = train_pred(X2_improved.iloc[:,2:10],Y2_new)
y1_new_proba = train_pred(X1_improved.iloc[:,2:10],Y1_new)

In [307]:
new_expected_utility1 = y1_new_proba.T @ u1_new[2:]
new_expected_utility2 = y2_new_proba.T @ u2_new[2:]

In [310]:
new_expected_utility1/len(X1_improved)

-0.12802193583611057

In [313]:
new_expected_utility2/len(X2_improved)

-0.14037994680436724